# Lib

In [1]:
from __future__ import annotations
# %pip uninstall mediocreatbest
%pip install --upgrade --force-reinstall \
    mediocreatbest@git+https://gist.github.com/player1537/3457b026ed6ef6696d758517f55a58df.git
try:
    from mediocreatbest import auto
except ImportError:
    %pip install --quiet --upgrade pip
    %pip install --upgrade --force-reinstall \
        mediocreatbest@git+https://gist.github.com/player1537/3457b026ed6ef6696d758517f55a58df.git
    from mediocreatbest import auto

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://gist.github.com/player1537/3457b026ed6ef6696d758517f55a58df.git to /tmp/pip-install-_n8rdspm/mediocreatbest_c62cdb0d31f8413e8150c4dcb0962d81
  Running command git clone --filter=blob:none --quiet https://gist.github.com/player1537/3457b026ed6ef6696d758517f55a58df.git /tmp/pip-install-_n8rdspm/mediocreatbest_c62cdb0d31f8413e8150c4dcb0962d81
  Resolved https://gist.github.com/player1537/3457b026ed6ef6696d758517f55a58df.git to commit e41d8d653f5889dd403fed60135cfcf0de585704
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mediocreatbest: filename=mediocreatbest-0.2.33-py3-none-any.whl size=16365 sha256=d298da127ce9515525cbe197011820785ae12f4ed973ff22e4b1290fd5a2457e
  Stored in directory: /tmp/pip-ephem-wheel-cache-x_yt8cny/wheels/cc/6d/dd/d4137c3485df0cb8773c136ecc231252590bf217127892aaa5


## Config

In [2]:
#@title Config { display-mode: "form" }
config = auto.types.SimpleNamespace()

/auto.pprint.pp config width=144

namespace()


## Spatial

In [3]:
#@title Spatial
class Spatial:
    Degree = auto.typing.NewType('Degree', float)
    Radian = auto.typing.NewType('Radian', float)
    Meter = auto.typing.NewType('Meter', float)
    Kilometer = auto.typing.NewType('Kilometer', float)

    def __new__(
        cls,
        *,
        lat: Degree,
        lng: Degree,
        alt: Meter,
    ) -> tuple[Kilometer, Kilometer, Kilometer]:
        Degree = cls.Degree
        Radian = cls.Radian
        Meter = cls.Meter
        Kilometer = cls.Kilometer

        # Thanks https://gis.stackexchange.com/a/4148

        #> Note that "Lat/Lon/Alt" is just another name for spherical coordinates, and
        #> phi/theta/rho are just another name for latitude, longitude, and altitude.
        #> :) (A minor difference: altitude is usually measured from the surface of the
        #> sphere; rho is measured from the center -- to convert, just add/subtract the
        #> radius of the sphere.)
        phi: Radian = auto.np.radians(lat)
        theta: Radian = auto.np.radians(lng)

        # Thanks https://en.wikipedia.org/wiki/Earth_radius
        #> A globally-average value is usually considered to be 6,371 kilometres (3,959 mi)
        rho: Kilometer = 6_371 + alt / 1000.0

        #> x = math.cos(phi) * math.cos(theta) * rho
        x: Kilometer = auto.np.cos(phi) * auto.np.cos(theta) * rho

        #> y = math.cos(phi) * math.sin(theta) * rho
        y: Kilometer = auto.np.cos(phi) * auto.np.sin(theta) * rho

        #> z = math.sin(phi) * rho # z is 'up'
        z: Kilometer = auto.np.sin(phi) * rho

        #> (Note there's some slightly arbitrary choices here in what each axis means...
        #> you might want 'y' to point at the north pole instead of 'z', for example.)

        # I do :)
        y, z = z, y

        return x, y, z

## with_exit_stack

In [4]:
#@title with_exit_stack
def with_exit_stack(func):
    def wrapper(*args, **kwargs):
        with auto.contextlib.ExitStack() as stack:
            return func(*args, stack=stack, **kwargs)
    return wrapper

# Buildings

In [5]:
config.app = auto.types.SimpleNamespace()
config.app.state = 'AK'

config.app.datadir = auto.pathlib.Path('/mnt/seenas2/data/model-america/data')
assert config.app.datadir.exists()

config.app.buildings = config.app.datadir / 'MAv1_CSVS' / f'{config.app.state}.csv'
assert config.app.buildings.exists()

## Unit

In [6]:
#@title Unit
@auto.functools.cache
def Unit():
    unit = auto.pint.UnitRegistry()

    return unit

_ = Unit()

## Location

In [7]:
#@title Location
@auto.dataclasses.dataclass(frozen=True, kw_only=True)
class Location:
    lat: float
    lng: float

    @classmethod
    @auto.mediocreatbest.doctest
    def parse(Location, s: str, /):
        r"""

        >>> parse(dict, "35.4963185/-88.468932")
        {'lat': 35.4963185, 'lng': -88.468932}

        """
        lat, lng = s.split('/')
        lat = float(lat)
        lng = float(lng)

        return Location(
            lat=lat,
            lng=lng,
        )


## Bounds

In [8]:
#@title Bounds
@auto.dataclasses.dataclass(frozen=True, kw_only=True)
class Bounds:
    sw: Location
    ne: Location

    def __post_init__(self):
        assert self.sw.lat <= self.ne.lat
        assert self.sw.lng <= self.ne.lng

## Building

In [9]:
#@title Building
@auto.dataclasses.dataclass(frozen=True, kw_only=True)
class Building:
    identifier: str
    centroid: Location
    footprint: list[Location]
    state: str
    volume: auto.pint.Quantity
    area: auto.pint.Quantity
    height: auto.pint.Quantity
    floors: int
    kind: str
    standard: str
    # center: Location
    # county: str

    @classmethod
    @auto.mediocreatbest.doctest
    def parse(Building, row: auto.df.Series, /) -> auto.typing.Self:
        # r"""

        # >>> parse(dict, {
        # ...     'ID': 6022005286564,
        # ...     'Centroid': '35.4963185/-88.468932',
        # ...     'Footprint2D': '35.49638/-88.468974_35.49638/-88.46889_35.496257/-88.46889_35.496257/-88.468974',
        # ...     'State_Abbr': 'TN',
        # ...     'Area': 5071.597932155534,
        # ...     'Area2D': 724.5139903079335,
        # ...     'Height': 20.68000066176,
        # ...     'NumFloors': 7.0,
        # ...     'WWR_surfaces': '.15_.15_.15_.15',
        # ...     'CZ': '3A',
        # ...     'BuildingType': 'MidriseApartment',
        # ...     'Standard': 'DOE-Ref-Pre-1980',
        # ...     'lat': 35.4963185,
        # ...     'lon': -88.468932,
        # ...     '.points': 'POINT (-88.468932 35.4963185)',
        # ...     'index_right': 0})
        # ... #doctest: +NORMALIZE_WHITESPACE
        # {'identifier': 6022005286564,
        # 'centroid': Location(lat=35.4963185, lng=-88.468932),
        # 'footprint': [Location(lat=35.49638, lng=-88.468974),
        #             Location(lat=35.49638, lng=-88.46889),
        #             Location(lat=35.496257, lng=-88.46889),
        #             Location(lat=35.496257, lng=-88.468974)],
        # 'state': 'TN',
        # 'volume': <Quantity(5071.59793, 'meter ** 3')>,
        # 'area': <Quantity(724.51399, 'meter ** 2')>,
        # 'height': <Quantity(20.6800007, 'meter')>,
        # 'floors': 7.0,
        # 'kind': 'MidriseApartment',
        # 'standard': 'DOE-Ref-Pre-1980',
        # }

        # """
        # 'center': Location(lat=35.4963185, lng=-88.468932),
        # ...     'City': 'Henderson'})

        unit = Unit()

        identifier = row['ID']
        centroid = Location.parse(row['Centroid'])
        footprint = [Location.parse(p) for p in row['Footprint2D'].split('_')]
        state = row['State_Abbr']
        volume = row['Area'] * unit.meter ** 3
        area = row['Area2D'] * unit.meter ** 2
        height = row['Height'] * unit.meter
        floors = row['NumFloors']
        kind = row['BuildingType']
        standard = row['Standard']
        # center = Location(
        #     lat=row['lat'],
        #     lng=row['lon'],
        # )
        # county = row['City']

        return Building(
            identifier=identifier,
            centroid=centroid,
            footprint=footprint,
            state=state,
            volume=volume,
            area=area,
            height=height,
            floors=floors,
            kind=kind,
            standard=standard,
            # center=center,
            # county=county,
        )

    @auto.functools.cached_property
    def url(building: Building, /) -> str:
        return (
            f'http://maps.google.com/maps'
            f'?z=12'
            f'&t=k'
            f'&q=loc:{building.centroid.lat}+{building.centroid.lng}'
        )

    @auto.functools.cached_property
    def bounds(building: Building, /) -> Bounds:
        n = e = float('-inf')
        s = w = float('inf')

        for location in building.footprint:
            n = max(n, location.lat)
            s = min(s, location.lat)
            w = min(w, location.lng)
            e = max(e, location.lng)

        return Bounds(
            sw=Location(lng=w, lat=s),
            ne=Location(lng=e, lat=n),
        )

    # @auto.functools.cached_property
    @property
    def osm(building: Building, /) -> dict[str, list[dict[str, list[str]]]]:
        root = config.app.things[building.county]
        path = root / 'ModelsOrig' / f'{building.identifier}' / f'{building.identifier}.osm'
        with path.open('r') as f:
            return Thing.parse(f.read())

    # @auto.functools.cached_property
    @property
    def idf(building: Building, /) -> dict[str, list[dict[str, list[str]]]]:
        root = config.app.things[building.county]
        path = root / 'ModelsOrig' / f'{building.identifier}' / f'{building.identifier}.idf'
        with path.open('r') as f:
            return Thing.parse(f.read())

## Buildings

In [10]:
#@title Buildings
class Buildings(auto.collections.UserList[Building]):
    @classmethod
    def parse(Buildings, fileobj, /, nrows=None):
        df = auto.pd.read_csv(
            fileobj,
            nrows=nrows,
        )

        buildings = Buildings()
        for _, row in auto.tqdm.notebook.tqdm(df.iterrows(), total=len(df)):
            buildings.append(Building.parse(row))

        return buildings

    @auto.functools.cached_property
    def knox(buildings_, /):
        buildings = Buildings()
        for building in buildings_:
            if building.county == 'Knox':
                buildings.append(building)

        return buildings

    @auto.functools.cached_property
    def henderson(buildings_, /):
        buildings = Buildings()
        for building in buildings_:
            if building.county == 'Henderson':
                buildings.append(building)

        return buildings

    @auto.functools.cached_property
    def bounds(buildings: Buildings, /) -> Bounds:
        n = e = float('-inf')
        s = w = float('inf')

        for building in buildings:
            bounds = building.bounds()
            n = max(n, bounds.ne.lat)
            s = min(s, bounds.sw.lat)
            w = min(w, bounds.sw.lng)
            e = max(e, bounds.ne.lng)

        return Bounds(
            sw=Location(lng=w, lat=s),
            ne=Location(lng=e, lat=n),
        )


# Testing

In [11]:
def scope():
    !ls -l {config.app.datadir}

    !ls -l {config.app.buildings}

/scope

total 0
drwxrwxr-x 1 raustin9 raustin9 111436 Oct  2 23:35 Counties_IDF
drwxrwxr-x 1 raustin9 raustin9 111436 Oct  3 10:05 Counties_OSM
drwxrwxr-x 1 raustin9 raustin9    612 Oct  2 17:26 MAv1_CSVS
-rw-r--r-- 1 raustin9 raustin9 31875690 Oct  2 17:17 /mnt/seenas2/data/model-america/data/MAv1_CSVS/AK.csv


In [12]:
buildings = Buildings.parse(config.app.buildings, nrows=10)
/print len(buildings)

  0%|          | 0/10 [00:00<?, ?it/s]

10


In [13]:
def scope():
    def scope():
        for building in buildings:
            print(building.centroid)

    scope()

/scope

Location(lat=61.25598322009571, lng=-150.02458711483254)
Location(lat=61.456785499999995, lng=-149.7233445)
Location(lat=61.458337967709284, lng=-149.71922760876606)
Location(lat=61.25204267682887, lng=-149.96062806058237)
Location(lat=61.357316024000006, lng=-149.89366773866664)
Location(lat=61.24774851385063, lng=-150.03094025225647)
Location(lat=61.246843, lng=-149.971444)
Location(lat=61.247634000000005, lng=-149.989395)
Location(lat=61.307521289115655, lng=-149.91605287515006)
Location(lat=61.458833999999996, lng=-149.72715)


# Generate OSPRAY

In [14]:
#@title save
def scope():
    # path = auto.pathlib.Path('/mnt/data') / 'buildings.zip'
    # if path.exists():
    #     path.unlink()

    # with auto.zipfile.ZipFile(path, 'w', compression=auto.zipfile.ZIP_DEFLATED) as arc:
    #     root = auto.zipfile.Path(arc)

    root = auto.pathlib.Path('/mnt/seenas2/data') / 'ct-buildings' / f'{config.app.state}'
    # root = auto.pathlib.Path('/mnt/data') / 'buildings'
    root.mkdir(exist_ok=True)

    with auto.contextlib.ExitStack() as stack:
        pbar = stack.enter_context( auto.tqdm.auto.tqdm(unit='building') )
        ebar = stack.enter_context( auto.tqdm.auto.tqdm(unit='error') )

        pbar.reset(total=len(buildings))
        for building in buildings:
            pbar.update()

            # path = root / f'{building.identifier}' / 'osm.json.gz'
            # path.parent.mkdir(exist_ok=True)
            # if not path.exists():
            #     try:
            #         osm = building.osm
            #     except KeyboardInterrupt:
            #         raise
            #     except:
            #         ebar.update()
            #         pass
            #     else:
            #         with auto.gzip.open(path, 'wb') as f:
            #             with auto.io.TextIOWrapper(f, encoding='utf-8') as f:
            #                 auto.json.dump(osm.sections, f)

            # path = root / f'{building.identifier}' / 'idf.json.gz'
            # path.parent.mkdir(exist_ok=True)

            # if path.exists() and path.stat().st_size < 1000:
            #     path.unlink()

            # if not path.exists():
            #     try:
            #         idf = building.idf
            #     except KeyboardInterrupt:
            #         raise
            #     except:
            #         ebar.update()
            #         pass
            #     else:
            #         with auto.gzip.open(path, 'wb') as f:
            #             with auto.io.TextIOWrapper(f, encoding='utf-8') as f:
            #                 auto.json.dump(idf.sections, f)

            path = root / f'{building.identifier}' / 'building.json'
            path.parent.mkdir(exist_ok=True)
            if not path.exists():
                with path.open('w') as f:
                    f.write(auto.json.dumps({
                        'identifier': building.identifier,
                        'centroid': {
                            'lat': building.centroid.lat,
                            'lng': building.centroid.lng,
                        },
                        'footprint': [
                            {
                                'lat': p.lat,
                                'lng': p.lng,
                            }
                            for p in building.footprint
                        ],
                        'state': building.state,
                        'volume': building.volume.m_as(Unit().meter ** 3),
                        'area': building.area.m_as(Unit().meter ** 2),
                        'height': building.height.m_as(Unit().meter),
                        'floors': building.floors,
                        'kind': building.kind,
                        'standard': building.standard,
                        # 'center': {
                        #     'lat': building.center.lat,
                        #     'lng': building.center.lng,
                        # },
                        # 'county': building.county,
                    }))

/scope


0building [00:00, ?building/s]

0error [00:00, ?error/s]

In [42]:
# Take the JSON and get footprint

def scope():
    path = f'/mnt/seenas2/data/ct-buildings/{config.app.state}'
    dir_list = auto.os.listdir(path)

    total_items = len(dir_list)
    # print(total_items)
    # return
    for building in auto.tqdm.auto.tqdm(dir_list):

        points = []
        building_height = 0
        with open(f'{path}/{building}/building.json') as fp:
            data = auto.json.load(fp)

            print(f'Building <{building}>')
            # print('\tFootprint:')
            for point in data["footprint"]:
                # print(f'\t\t({point["lat"]}, {point["lng"]})')
                x, y, z = Spatial(
                    lat=point["lat"],
                    lng=point["lng"],
                    alt=0
                )
                points.append(auto.numpy.array([x,y,z]))
                # points.append(auto.numpy.array([x * 1000,y * 1000,z * 1000]))
                # points.append((x,y,z))
                # print(
                #     f'\t\t> Spatial: ({x}, {y}, {z})'
                # )
            # print(f'\tHeight: {data["height"]}')
            building_height = float(data["height"])
            # print()
        
        points = auto.numpy.array(points)
        # Get top face of building
        def normalize_vector(point):
            # Convert point to a numpy array for easy calculations
            vector = auto.numpy.array(point)
    
            # Calculate the magnitude of the vector
            magnitude = auto.numpy.linalg.norm(vector)
    
            # Check if the magnitude is not zero to avoid division by zero
            if magnitude == 0:
                raise ValueError("Cannot normalize a zero vector.")
    
            # Normalize the vector
            normalized_vector = vector / magnitude / 100
            return normalized_vector
        
        normal = normalize_vector(points[0])
        vertical = normal
        # vertical = normal * (building_height / 350)
        print(f'Normal {normal}. Height: {building_height} -> Vertical: {vertical}')

        # find points for top of building
        roof_verts = []
        # print("Points")
        for p in points:
            # print(p)
            roof_verts.append(p + vertical)


        roof_verts = auto.numpy.array(roof_verts)

        faces = []
        num_vertices = len(points)
        for i in range(num_vertices):
            next_index = (i+1) % num_vertices
            faces.append([i, next_index, num_vertices + next_index])  # Triangle 1
            faces.append([i, num_vertices + next_index, num_vertices + i])  # Triangle 2

        for p in points:
            p[2] = -p[2]
            p[1] = -p[1]
            p[0] = -p[0]
            # print(p[2])
            # return
        for p in roof_verts:
            p[2] = -p[2]
            p[1] = -p[1]
            p[0] = -p[0]
            # print(p[2])
            # return
        vertices = auto.numpy.vstack((points, roof_verts))
        mesh = auto.trimesh.Trimesh(vertices=vertices, faces=faces)
        # mesh.export('building.obj')

        # print(mesh.vertices)
        verts = auto.numpy.array(mesh.vertices, dtype='f4').flatten()
        indices = auto.numpy.array(mesh.faces, dtype='u4').flatten()
        print(verts)
        print(indices)
        # for i in range(len(indices)):
        #     print(f'{indices[i]}  |  ', end='')
        #     if (i+1) % 3 == 0:
        #         print()
        # light = auto.trimesh.scene.lighting.DirectionalLight()
        # light.intensity = 10
        # scene = auto.trimesh.Scene([mesh], lights=[light])
        # scene.show(viewer='notebook')

        # all_vertices = auto.numpy.concatenate([all_vertices, auto.numpy.array(mesh.vertices).flatten()])
        # all_indices = auto.numpy.concatenate([all_indices, auto.numpy.array(mesh.faces).flatten()])

        with open(f'/mnt/seenas2/data/ct-buildings/gen/{config.app.state}/{building}.mesh.vec3f[].vertex.position.bin', 'wb') as f:
            f.write(verts.tobytes())
        with open(f'/mnt/seenas2/data/ct-buildings/gen/{config.app.state}/{building}.mesh.vec3ui[].vertex.index.bin', 'wb') as f:
            f.write(indices.tobytes())

        # break




/scope

  0%|          | 0/10 [00:00<?, ?it/s]

Building <9021005053130>
Normal [-0.00416572  0.00876777 -0.0024027 ]. Height: 27.444254647983406 -> Vertical: [-0.00416572  0.00876777 -0.0024027 ]
[ 2653.9778 -5585.948   1530.7588  2653.9788 -5585.9478  1530.7594
  2653.977  -5585.9478  1530.7623  2653.9832 -5585.944   1530.7657
  2653.9883 -5585.944   1530.7571  2653.9812 -5585.948   1530.7529
  2653.982  -5585.957   1530.7612  2653.983  -5585.9565  1530.7618
  2653.9812 -5585.9565  1530.7646  2653.9873 -5585.9526  1530.7682
  2653.9924 -5585.9526  1530.7595  2653.9854 -5585.957   1530.7554]
[ 0  1  7  0  7  6  1  2  8  1  8  7  2  3  9  2  9  8  3  4 10  3 10  9
  4  5 11  4 11 10  5  0  6  5  6 11]
Building <9021005053131>
Normal [-0.00412647  0.00878457 -0.00240905]. Height: 44.7082232820868 -> Vertical: [-0.00412647  0.00878457 -0.00240905]
[ 2628.977  -5596.6494  1534.8086  2628.971  -5596.6514  1534.812
  2628.9724 -5596.6494  1534.8159  2628.9785 -5596.6475  1534.8125
  2628.9812 -5596.658   1534.811   2628.975  -5596.66    